In [94]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import KNNImputer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from numpy import log1p
from xgboost import XGBClassifier

In [95]:
data = pd.read_csv("train.csv")
data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [97]:
data.nunique()

PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [98]:
data.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [99]:
data['Transported'] = data['Transported'].astype(int)
data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0


In [100]:
data['CryoSleep'] = data['CryoSleep'].map({True: 1, False: 0})
data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0
8689,9278_01,Earth,1.0,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0
8690,9279_01,Earth,0.0,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1
8691,9280_01,Europa,0.0,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0


In [101]:
data['VIP'] = data['VIP'].map({True: 1, False: 0})

In [102]:
data["Cabin_0idx"] = data["Cabin"].apply(lambda x: x.split("/")[0] if pd.notnull(x) else np.nan)
data["Cabin_1idx"] = data["Cabin"].apply(lambda x: x.split("/")[1] if pd.notnull(x) else np.nan)
data["Cabin_2idx"] = data["Cabin"].apply(lambda x: x.split("/")[2] if pd.notnull(x) else np.nan)
data.drop(columns=["Cabin"], axis=1, inplace=True)
data

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Cabin_0idx,Cabin_1idx,Cabin_2idx
0,0001_01,Europa,0.0,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,B,0,P
1,0002_01,Earth,0.0,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,F,0,S
2,0003_01,Europa,0.0,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,A,0,S
3,0003_02,Europa,0.0,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,A,0,S
4,0004_01,Earth,0.0,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,A,98,P
8689,9278_01,Earth,1.0,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,G,1499,S
8690,9279_01,Earth,0.0,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,G,1500,S
8691,9280_01,Europa,0.0,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,E,608,S


In [29]:
data["Cabin_0idx"].unique()

array(['B', 'F', 'A', 'G', nan, 'E', 'D', 'C', 'T'], dtype=object)

In [103]:
data["Cabin_2idx"] = data["Cabin_2idx"].map({"P": 1, "S": 0})
data

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Cabin_0idx,Cabin_1idx,Cabin_2idx
0,0001_01,Europa,0.0,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,B,0,1.0
1,0002_01,Earth,0.0,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,F,0,0.0
2,0003_01,Europa,0.0,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,A,0,0.0
3,0003_02,Europa,0.0,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,A,0,0.0
4,0004_01,Earth,0.0,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,F,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,A,98,1.0
8689,9278_01,Earth,1.0,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,G,1499,0.0
8690,9279_01,Earth,0.0,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,G,1500,0.0
8691,9280_01,Europa,0.0,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,E,608,0.0


In [104]:
data["isRich"] = data["Cabin_0idx"].map(
    lambda x: 1 if x in ["A", "B"] else (0 if pd.notna(x) else np.nan)
)
data

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Cabin_0idx,Cabin_1idx,Cabin_2idx,isRich
0,0001_01,Europa,0.0,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,B,0,1.0,1.0
1,0002_01,Earth,0.0,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,F,0,0.0,0.0
2,0003_01,Europa,0.0,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,A,0,0.0,1.0
3,0003_02,Europa,0.0,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,A,0,0.0,1.0
4,0004_01,Earth,0.0,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,F,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,A,98,1.0,1.0
8689,9278_01,Earth,1.0,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,G,1499,0.0,0.0
8690,9279_01,Earth,0.0,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,G,1500,0.0,0.0
8691,9280_01,Europa,0.0,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,E,608,0.0,0.0


In [105]:
data['Cabin_0idx'].fillna('Unknown', inplace=True)
# Create encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform
encoded_array = encoder.fit_transform(data[["Cabin_0idx"]])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(["Cabin_0idx"]))

# Combine with original data (optional: drop old column)
data = pd.concat([data.drop("Cabin_0idx", axis=1), encoded_df], axis=1)
data

C:\Users\PcKala.org\AppData\Local\Temp\ipykernel_3568\3180721499.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Cabin_0idx'].fillna('Unknown', inplace=True)


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,...,isRich,Cabin_0idx_A,Cabin_0idx_B,Cabin_0idx_C,Cabin_0idx_D,Cabin_0idx_E,Cabin_0idx_F,Cabin_0idx_G,Cabin_0idx_T,Cabin_0idx_Unknown
0,0001_01,Europa,0.0,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0002_01,Earth,0.0,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0003_01,Europa,0.0,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0003_02,Europa,0.0,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0004_01,Earth,0.0,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8689,9278_01,Earth,1.0,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8690,9279_01,Earth,0.0,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8691,9280_01,Europa,0.0,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [106]:
data['Destination'].fillna('Unknown', inplace=True)
# Create encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform
encoded_array = encoder.fit_transform(data[["Destination"]])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(["Destination"]))

# Combine with original data (optional: drop old column)
data_encoded = pd.concat([data.drop("Destination", axis=1), encoded_df], axis=1)
data_encoded

C:\Users\PcKala.org\AppData\Local\Temp\ipykernel_3568\3559490150.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Destination'].fillna('Unknown', inplace=True)


,PassengerId,HomePlanet,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,Cabin_0idx_D,Cabin_0idx_E,Cabin_0idx_F,Cabin_0idx_G,Cabin_0idx_T,Cabin_0idx_Unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown
0,0001_01,Europa,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0002_01,Earth,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0003_01,Europa,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0003_02,Europa,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0004_01,Earth,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
8689,9278_01,Earth,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8690,9279_01,Earth,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8691,9280_01,Europa,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [107]:
data_encoded['HomePlanet'].fillna('Unknown', inplace=True)
# Create encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform
encoded_array = encoder.fit_transform(data_encoded[["HomePlanet"]])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(["HomePlanet"]))

# Combine with original data (optional: drop old column)
data_encoded_2 = pd.concat([data_encoded.drop("HomePlanet", axis=1), encoded_df], axis=1)
data_encoded_2

C:\Users\PcKala.org\AppData\Local\Temp\ipykernel_3568\3340952290.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_encoded['HomePlanet'].fillna('Unknown', inplace=True)


,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,Cabin_0idx_T,Cabin_0idx_Unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown
0,0001_01,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0002_01,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0003_01,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0003_02,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0004_01,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8689,9278_01,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
8690,9279_01,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8691,9280_01,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
#gender_csv = pd.read_csv("https://archive.ics.uci.edu/static/public/591/data.csv")
#gender_csv.columns = ['Name', 'Gender','Count','Probabilty']
#gender_csv['Name'] = gender_csv['Name'].str.strip().str.lower()
# Extract and normalize first name
#data_encoded_2['Name'] = data_encoded_2['Name'].str.split().str[0].str.lower().str.strip()
# Create a name-to-gender dictionary
#name_gender_dict = dict(zip(gender_csv['Name'], gender_csv['Gender']))

# Map gender
#data_encoded_2['gender'] = data_encoded_2['Name'].map(name_gender_dict)

# Optional: fill missing values
#data_encoded_2['gender'].fillna('unknown', inplace=True)

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,Cabin_2idx,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown,gender
0,0001_01,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,maham,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,F
1,0002_01,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,juanna,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,F
2,0003_01,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,altark,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,NaN
3,0003_02,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,solam,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,NaN
4,0004_01,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,willy,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,gravior,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN
8689,9278_01,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,kurta,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,M
8690,9279_01,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,fayey,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,NaN
8691,9280_01,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,celeon,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN


In [ ]:
#gender_csv = pd.read_csv("https://archive.ics.uci.edu/static/public/591/data.csv")

,Name,Gender,Count,Probability
0,James,M,5304407,1.451679e-02
1,John,M,5260831,1.439753e-02
2,Robert,M,4970386,1.360266e-02
3,Michael,M,4579950,1.253414e-02
4,William,M,4226608,1.156713e-02
...,...,...,...,...
147264,Zylenn,M,1,2.736740e-09
147265,Zymeon,M,1,2.736740e-09
147266,Zyndel,M,1,2.736740e-09
147267,Zyshan,M,1,2.736740e-09


In [108]:
data_encoded_2

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,Cabin_0idx_T,Cabin_0idx_Unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown
0,0001_01,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0002_01,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0003_01,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0003_02,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0004_01,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8689,9278_01,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
8690,9279_01,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8691,9280_01,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [109]:
cor_with_transported = data_encoded_2.corr(numeric_only=True)['Transported'].drop('Transported')
cor_with_transported

CryoSleep                    0.468645
Age                         -0.075026
VIP                         -0.037650
RoomService                 -0.244611
FoodCourt                    0.046566
ShoppingMall                 0.010141
Spa                         -0.221131
VRDeck                      -0.207075
Cabin_2idx                  -0.103775
isRich                       0.127930
Cabin_0idx_A                -0.002623
Cabin_0idx_B                 0.144733
Cabin_0idx_C                 0.108193
Cabin_0idx_D                -0.034046
Cabin_0idx_E                -0.097965
Cabin_0idx_F                -0.087753
Cabin_0idx_G                 0.016269
Cabin_0idx_T                -0.014568
Cabin_0idx_Unknown          -0.000340
Destination_55 Cancri e      0.108722
Destination_PSO J318.5-22    0.000092
Destination_TRAPPIST-1e     -0.094700
Destination_Unknown          0.000547
HomePlanet_Earth            -0.169019
HomePlanet_Europa            0.176916
HomePlanet_Mars              0.019544
HomePlanet_U

In [47]:
data_encoded_2

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,Cabin_0idx,Cabin_2idx,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown
0,0001_01,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0002_01,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0003_01,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0003_02,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0004_01,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8689,9278_01,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
8690,9279_01,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8691,9280_01,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [111]:
data_encoded_2.drop(columns=['PassengerId', 'Name'], inplace=True)

In [116]:
# Find numerical column
num_cols = data_encoded_2.select_dtypes(include=['float64','int64']).columns.tolist()
print(num_cols)

# Define imputers
num_imputer = KNNImputer(n_neighbors=5, weights='distance')


['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'Cabin_2idx', 'isRich', 'Cabin_0idx_A', 'Cabin_0idx_B', 'Cabin_0idx_C', 'Cabin_0idx_D', 'Cabin_0idx_E', 'Cabin_0idx_F', 'Cabin_0idx_G', 'Cabin_0idx_T', 'Cabin_0idx_Unknown', 'Destination_55 Cancri e', 'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'Destination_Unknown', 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars', 'HomePlanet_Unknown']


In [117]:
num_imputer = KNNImputer(n_neighbors=5)
preprocessed_data = num_imputer.fit_transform(data_encoded_2[num_cols])

# Convert to DataFrame
preprocessed_df = pd.DataFrame(preprocessed_data, columns=num_cols)
preprocessed_df['Transported'] = data_encoded_2['Transported'].values

In [118]:
preprocessed_df

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin_2idx,...,Cabin_0idx_T,Cabin_0idx_Unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown
0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8689,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
8690,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8691,0.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [119]:
x = preprocessed_df.drop("Transported",axis=1)
y = preprocessed_df["Transported"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [120]:
param_grid = {
    'hidden_layer_sizes': [(64, 32), (128, 64), (64, 64, 32)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.001, 0.01]
}

grid = GridSearchCV(MLPRegressor(max_iter=500), param_grid, scoring='neg_mean_squared_error', cv=5)
grid.fit(x_train, y_train)

print("Best parameters:", grid.best_params_)

d:\python-projects\kaggle-datasets\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:787: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Best parameters: {'alpha': 0.0001, 'hidden_layer_sizes': (128, 64), 'learning_rate_init': 0.01}


In [121]:
model = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    alpha=0.001,
    learning_rate_init=0.0001,
    max_iter=500,  # increase from default 200
    random_state=42
)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7619321449108684
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.66      0.73       861
           1       0.72      0.86      0.79       878

    accuracy                           0.76      1739
   macro avg       0.77      0.76      0.76      1739
weighted avg       0.77      0.76      0.76      1739



In [122]:
best_params = {
    'n_estimators': int(np.int64(300)),
    'min_samples_split': 10,
    'min_samples_leaf': 1,
    'max_features': 'log2',
    'max_depth': int(np.int64(20)),
    'bootstrap': True,
    'random_state': 42  # Always fix random state for reproducibility
}

model = RandomForestClassifier(**best_params)
model.fit(x_train, y_train)

# Predict on test set
y_pred = model.predict(x_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7906843013225991
Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.80      0.79       861
           1       0.80      0.78      0.79       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79      0.79      1739
weighted avg       0.79      0.79      0.79      1739



In [58]:
rf = RandomForestClassifier(random_state=42)

param_dist = {
    'n_estimators': np.arange(50, 301, 50),
    'max_depth': [None] + list(np.arange(5, 31, 5)),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],  # no 'auto'
    'bootstrap': [True, False]
}

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,                 # Number of parameter settings sampled
    scoring='roc_auc',         # Optimize for ROC AUC
    cv=5,                     # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1                 # Use all CPU cores
)

# Run hyperparameter search
random_search.fit(x_train, y_train)

# Best parameters
print("Best Hyperparameters:", random_search.best_params_)

# Evaluate on test set
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(x_test)
y_proba = best_rf.predict_proba(x_test)[:, 1]

print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best Hyperparameters: {'n_estimators': np.int64(200), 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': np.int64(10), 'bootstrap': False}
              precision    recall  f1-score   support

           0       0.81      0.74      0.77       861
           1       0.76      0.83      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739



In [89]:
param_grid = {
    'penalty': ['l1', 'l2'],  # 'elasticnet' or 'none' if solver supports
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],  # use saga for large/elasticnet
    'max_iter': [100, 200, 500]
}
logreg = LogisticRegression(random_state=42)

grid_search = GridSearchCV(
    estimator=logreg,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',  # or 'roc_auc', 'f1' for imbalanced data
    n_jobs=-1,
    verbose=1
)

grid_search.fit(x_train, y_train)
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best Parameters: {'C': 0.1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score: 0.7956589379826118


In [72]:
logistic_model = LogisticRegression(
    C=0.1,
    penalty='l1',
    solver='liblinear',
    max_iter=100
)
logistic_model.fit(x_train,y_train)
y_pred = logistic_model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.777458309373203


In [153]:
test = pd.read_csv("test.csv")
test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [154]:
test['CryoSleep'] = test['CryoSleep'].map({True: 1, False: 0})
test['VIP'] = test['VIP'].map({True: 1, False: 0})
test["Cabin_0idx"] = test["Cabin"].apply(lambda x: x.split("/")[0] if pd.notnull(x) else np.nan)
test["Cabin_1idx"] = test["Cabin"].apply(lambda x: x.split("/")[1] if pd.notnull(x) else np.nan)
test["Cabin_2idx"] = test["Cabin"].apply(lambda x: x.split("/")[2] if pd.notnull(x) else np.nan)
test.drop(columns=["Cabin"], axis=1, inplace=True)


In [155]:
test["isRich"] = test["Cabin_0idx"].map(
    lambda x: 1 if x in ["A", "B"] else (0 if pd.notna(x) else np.nan)
)
test

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Cabin_0idx,Cabin_1idx,Cabin_2idx,isRich
0,0013_01,Earth,1.0,TRAPPIST-1e,27.0,0.0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,G,3,S,0.0
1,0018_01,Earth,0.0,TRAPPIST-1e,19.0,0.0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,F,4,S,0.0
2,0019_01,Europa,1.0,55 Cancri e,31.0,0.0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,C,0,S,0.0
3,0021_01,Europa,0.0,TRAPPIST-1e,38.0,0.0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,C,1,S,0.0
4,0023_01,Earth,0.0,TRAPPIST-1e,20.0,0.0,10.0,0.0,635.0,0.0,0.0,Brence Harperez,F,5,S,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,1.0,TRAPPIST-1e,34.0,0.0,0.0,0.0,0.0,0.0,0.0,Jeron Peter,G,1496,S,0.0
4273,9269_01,Earth,0.0,TRAPPIST-1e,42.0,0.0,0.0,847.0,17.0,10.0,144.0,Matty Scheron,NaN,NaN,NaN,NaN
4274,9271_01,Mars,1.0,55 Cancri e,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,D,296,P,0.0
4275,9273_01,Europa,0.0,NaN,NaN,0.0,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,D,297,P,0.0


In [156]:
test['Cabin_0idx'].fillna('Unknown', inplace=True)
# Create encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform
encoded_array = encoder.fit_transform(test[["Cabin_0idx"]])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(["Cabin_0idx"]))

# Combine with original data (optional: drop old column)
test = pd.concat([test.drop("Cabin_0idx", axis=1), encoded_df], axis=1)
test

C:\Users\PcKala.org\AppData\Local\Temp\ipykernel_3568\2579734985.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Cabin_0idx'].fillna('Unknown', inplace=True)


,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,...,isRich,Cabin_0idx_A,Cabin_0idx_B,Cabin_0idx_C,Cabin_0idx_D,Cabin_0idx_E,Cabin_0idx_F,Cabin_0idx_G,Cabin_0idx_T,Cabin_0idx_Unknown
0,0013_01,Earth,1.0,TRAPPIST-1e,27.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0018_01,Earth,0.0,TRAPPIST-1e,19.0,0.0,0.0,9.0,0.0,2823.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0019_01,Europa,1.0,55 Cancri e,31.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0021_01,Europa,0.0,TRAPPIST-1e,38.0,0.0,0.0,6652.0,0.0,181.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0023_01,Earth,0.0,TRAPPIST-1e,20.0,0.0,10.0,0.0,635.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,1.0,TRAPPIST-1e,34.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4273,9269_01,Earth,0.0,TRAPPIST-1e,42.0,0.0,0.0,847.0,17.0,10.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4274,9271_01,Mars,1.0,55 Cancri e,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4275,9273_01,Europa,0.0,NaN,NaN,0.0,0.0,2680.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [157]:
test["Cabin_2idx"] = test["Cabin_2idx"].map({"P": 1, "S": 0})

In [158]:
test.drop("Cabin_1idx",axis=1,inplace=True)
test

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,...,isRich,Cabin_0idx_A,Cabin_0idx_B,Cabin_0idx_C,Cabin_0idx_D,Cabin_0idx_E,Cabin_0idx_F,Cabin_0idx_G,Cabin_0idx_T,Cabin_0idx_Unknown
0,0013_01,Earth,1.0,TRAPPIST-1e,27.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0018_01,Earth,0.0,TRAPPIST-1e,19.0,0.0,0.0,9.0,0.0,2823.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0019_01,Europa,1.0,55 Cancri e,31.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0021_01,Europa,0.0,TRAPPIST-1e,38.0,0.0,0.0,6652.0,0.0,181.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0023_01,Earth,0.0,TRAPPIST-1e,20.0,0.0,10.0,0.0,635.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,1.0,TRAPPIST-1e,34.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4273,9269_01,Earth,0.0,TRAPPIST-1e,42.0,0.0,0.0,847.0,17.0,10.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4274,9271_01,Mars,1.0,55 Cancri e,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4275,9273_01,Europa,0.0,NaN,NaN,0.0,0.0,2680.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [159]:
test['Destination'].fillna('Unknown', inplace=True)
# Create encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform
encoded_array = encoder.fit_transform(test[["Destination"]])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(["Destination"]))

# Combine with original data (optional: drop old column)
data_encoded_test = pd.concat([test.drop("Destination", axis=1), encoded_df], axis=1)
data_encoded_test

C:\Users\PcKala.org\AppData\Local\Temp\ipykernel_3568\817190954.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test['Destination'].fillna('Unknown', inplace=True)


,PassengerId,HomePlanet,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,Cabin_0idx_D,Cabin_0idx_E,Cabin_0idx_F,Cabin_0idx_G,Cabin_0idx_T,Cabin_0idx_Unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown
0,0013_01,Earth,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0018_01,Earth,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0019_01,Europa,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0021_01,Europa,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0023_01,Earth,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4273,9269_01,Earth,0.0,42.0,0.0,0.0,847.0,17.0,10.0,144.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4274,9271_01,Mars,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4275,9273_01,Europa,0.0,NaN,0.0,0.0,2680.0,0.0,0.0,523.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [160]:
data_encoded_test['HomePlanet'].fillna('Unknown', inplace=True)
# Create encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform
encoded_array = encoder.fit_transform(data_encoded_test[["HomePlanet"]])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(["HomePlanet"]))

# Combine with original data (optional: drop old column)
data_encoded_2_test = pd.concat([data_encoded_test.drop("HomePlanet", axis=1), encoded_df], axis=1)
data_encoded_2_test

C:\Users\PcKala.org\AppData\Local\Temp\ipykernel_3568\3113395471.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_encoded_test['HomePlanet'].fillna('Unknown', inplace=True)


,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,Cabin_0idx_T,Cabin_0idx_Unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown
0,0013_01,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0018_01,0.0,19.0,0.0,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0019_01,1.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0021_01,0.0,38.0,0.0,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0023_01,0.0,20.0,0.0,10.0,0.0,635.0,0.0,0.0,Brence Harperez,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,Jeron Peter,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4273,9269_01,0.0,42.0,0.0,0.0,847.0,17.0,10.0,144.0,Matty Scheron,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4274,9271_01,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4275,9273_01,0.0,NaN,0.0,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [161]:
data_encoded_2_test.drop(columns=['PassengerId', 'Name'], inplace=True)

In [162]:
# Find numerical column
num_cols = data_encoded_2_test.columns
# Define imputers
num_imputer = KNNImputer(n_neighbors=5, weights='distance')

In [163]:
# Use imputer
preprocessed_data = num_imputer.fit_transform(data_encoded_2_test)
# Convert to DataFrame
preprocessed_df_test = pd.DataFrame(preprocessed_data, columns=num_cols)
preprocessed_df_test

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin_2idx,isRich,...,Cabin_0idx_T,Cabin_0idx_Unknown,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_Unknown,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_Unknown
0,1.0,27.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.0,19.000000,0.0,0.0,9.0,0.0,2823.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,1.0,31.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,38.000000,0.0,0.0,6652.0,0.0,181.0,585.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,20.000000,0.0,10.0,0.0,635.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1.0,34.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4273,0.0,42.000000,0.0,0.0,847.0,17.0,10.0,144.0,0.195873,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4274,1.0,35.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4275,0.0,26.347682,0.0,0.0,2680.0,0.0,0.0,523.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [164]:
y_pred = model.predict(preprocessed_df_test)
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Transported": y_pred
})
submission

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,0
...,...,...
4272,9266_02,1
4273,9269_01,0
4274,9271_01,1
4275,9273_01,1


In [167]:
submission["Transported"] = submission["Transported"].map({1:True,0:False})

In [168]:
submission.to_csv("submission.csv",index=False)